# wps_CI_on_demand_with_indices

#### wps_CI is a process that runs the [ci.netcdf.wrapper](https://github.com/pacificclimate/ClimDown/blob/master/R/CI.R#L235) (Climate Imprint) function of the [ClimDown](https://cran.r-project.org/web/packages/ClimDown/index.html) package using the [`chickadee`](https://github.com/pacificclimate/chickadee) DACCS bird. This notebook uses either the PNWNAmet daily gridded observations or the CMIP6 downscaled data as the `gcm_file` parameter and the PRISM monthly climatologies as the `obs_file` parameter. By supplying climatologies, this skips the computation of the climatologies during Climate Imprint. After running CI, the output can be passed to the Ouranos [`finch`](https://github.com/bird-house/finch/tree/master) bird to compute climate indices. This notebook uses an interactive map to set up inputs to the birds and run them, and its setup process is detailed in the `map.py` module.

I. Import `map` module and its functions/variables.

In [1]:
import map
from map import *

II. Check `url`s used to connect to chickadee/finch.

In [2]:
# NBVAL_IGNORE_OUTPUT
print(f"Using chickadee on {chickadee_url}")
print(f"Using finch on {finch_url}")

Using chickadee on http://docker-dev03.pcic.uvic.ca:30102
Using finch on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/finch/wps


III. Display help for individual processes by using the ? command (ex. bird.process?). We can use the docstring to ensure we provide the appropriate parameters.

In [3]:
# NBVAL_IGNORE_OUTPUT
chickadee.ci?

Signature:
chickadee.ci(
    gcm_file,
    obs_file,
    gcm_varname=None,
    obs_varname=None,
    num_cores='4',
    loglevel='INFO',
    units_bool=True,
    n_pr_bool=True,
    tasmax_units='celsius',
    tasmin_units='celsius',
    pr_units='kg m-2 d-1',
    max_gb=1.0,
    start_date=datetime.date(1971, 1, 1),
    end_date=datetime.date(2005, 12, 31),
    out_file=None,
)
Docstring:
Climate Imprint (CI) downscaling

Parameters
----------
gcm_file : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Filename of GCM simulations
obs_file : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    Filename of high-res gridded historical observations
out_file : string
    Filename to create with the climate imprint outputs
num_cores : {'1', '2', '3', '4'}positiveInteger
    The number of cores to use for parallel execution
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level
units_boo

In [4]:
# NBVAL_IGNORE_OUTPUT
finch.tx_max?

Signature:
finch.tx_max(
    tasmax=None,
    freq='YS',
    month=None,
    season=None,
    check_missing='any',
    missing_options=None,
    cf_compliance='warn',
    data_validation='raise',
    variable=None,
    output_name=None,
    output_format='netcdf',
    csv_precision=None,
    output_formats=None,
)
Docstring:
The maximum value of daily maximum temperature.

Parameters
----------
tasmax : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
    NetCDF Files or archive (tar/zip) containing netCDF files. Maximum surface temperature.
freq : {'YS', 'MS', 'QS-DEC', 'AS-JUL'}string
    Resampling frequency.
month : {'1', '2', '3', '4', '5', '6', '7', '8', '9', '10', ...}integer
    Months of the year over which to compute indicator.
season : {'DJF', 'MAM', 'JJA', 'SON'}string
    Climatological season over which to compute indicator.
check_missing : {'any', 'wmo', 'pct', 'at_least_n', 'skip', 'from_context'}string
    Method used to determine which a

IV. Load the interactive map to initialize the inputs. These inputs are the <b>center point</b> of the 2.5 degree by 2.5 degree subdomain of the target region and 3 degree by 3 degree subdomain of the GCM region, the <b>climate variable</b> from the GCM input to downscale, the <b>dataset</b> to use as the GCM input (where choosing the CMIP6 data allows one to further select the <b>downscaling technique</b>, the <b>model</b>, the <b>CanESM5 run</b> if applicable, and the <b>emissions scenario</b>), and the <b>climatological period</b> of the target data. Some inputs are obtained from the [THREDDS data server](https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/thredds/catalog.html). Once the inputs are initialized, you can click the `Run Chickadee` button to start the CI process, and afterwards, click the `Run Finch` button to compute the climate index associated with the CI output.

In [5]:
AppLayout(center = m, right_sidebar = control_box, align_items = 'center')

AppLayout(children=(Box(children=(Text(value='', placeholder=''), Text(value='', description='Center:', placeh…

V. Run this cell to display bird progress.

In [6]:
ipydisplay.display(output_widget)

Output()

VI. Check statuses of birds and store outputs as `netCDF4.Dataset` objects if possible. `map.<bird>_output` is used to ensure that the updated value rather than the default `None` value is passed to these functions.

In [25]:
get_bird_output(map.chickadee_output)
if map.chickadee_output.isComplete():
    chickadee_output_ds = bird_output_to_dataset(map.chickadee_output)
    print(chickadee_output_ds)

Process status: ProcessSucceeded
Link to process output: https://docker-dev03.pcic.uvic.ca/wpsoutputs/6f8488fa-c393-11ee-9b94-0242ac120017/tasmax_PNWNAmet_197101-200012_on-demand.nc
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format DAP2):
    DODS_EXTRA.Unlimited_Dimension: time
    dimensions(sizes): time(24837), lat(301), lon(301)
    variables(dimensions): float64 lon(lon), float64 lat(lat), float64 time(time), float32 tasmax(time, lat, lon)
    groups: 


In [26]:
get_bird_output(map.finch_output)
if map.finch_output.isComplete():
    finch_output_ds = bird_output_to_dataset(map.finch_output)
    print(finch_output_ds)

Process status: ProcessSucceeded
Link to process output: https://docker-dev03.pcic.uvic.ca/wpsoutputs/finch/b19389be-c396-11ee-93d5-0242ac12000b/tx_max_ys.nc
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format DAP2):
    DODS_EXTRA.Unlimited_Dimension: time
    climateindex_package_id: https://github.com/Ouranosinc/xclim
    product: derived climate index
    contact: Canadian Centre for Climate Services
    institution: Canadian Centre for Climate Services (CCCS)
    institute_id: CCCS
    dimensions(sizes): time(68), lat(301), lon(301)
    variables(dimensions): float32 time(time), float64 lat(lat), float64 lon(lon), float32 tx_max(time, lat, lon)
    groups: 
